In [ ]:
import numpy as np
import os
import pandas as pd

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from sklearn.mixture import GaussianMixture
from scipy.stats import norm

import sys
sys.path.append('../../src/')
from config import *

In [ ]:
import mixem
import scipy
from scipy.special import gammaln

class Poisson(mixem.distribution.Distribution):

    def __init__(self, lambda_) -> None:
        super().__init__()
        self.lambda_ = lambda_

    def pmf(self, x):
        return np.exp( np.log(self.lambda_)*x  - self.lambda_ - gammaln(x+1) )

    def estimate_parameters(self, data, weights):
        prob = self.pmf(data)
        self.lambda_ = np.sum(prob*data)/np.sum(prob)

    def log_density(self, data):
        return np.log(self.pmf(data))

    def __repr__(self):
        return "Poisson[lambda={lbd:.4g}]".format(lbd=self.lambda_)

class logGMM:

    def __init__(self) -> None:
        
        self.weights = [] 
        self.distributions = []
        self.data = []

    def fit_logGMM(self, data):
        data = np.array(data)
        self.data = data[data>0.0]

        # Fit and plot log gaussian mixture
        self.weights, self.distributions, log_likelihood = mixem.em(self.data, [
            mixem.distribution.LogNormalDistribution(mu=-6, sigma=2.4),
            mixem.distribution.LogNormalDistribution(mu=-2.7, sigma=0.1),
            # mixem.distribution.NormalDistribution(mu=300, sigma=50),
        ], initial_weights=[0.8, 0.2], progress_callback=None)

    def get_pdf(self, x_axis):
        return mixem.probability(x_axis, self.weights, self.distributions)

    def find_midpoint(self, interval=[1e-3, 1e-1]):
        x_search = np.linspace(interval[0], interval[1], 10000)

        pdf1 = np.exp(self.distributions[0].log_density(x_search))
        pdf2 = np.exp(self.distributions[1].log_density(x_search))

        x_max_1 = np.argmax(pdf1)
        x_max_2 = np.argmax(pdf2)

        if x_max_1 < x_max_2:
            interval = [x_max_1, x_max_2]
        else:
            interval = [x_max_2, x_max_1]

        pdf_diff = np.abs(pdf1-pdf2)[interval[0]:interval[1]]
        if pdf_diff.size >0:
            return x_search[np.argmin(pdf_diff)]
        else:
            return 0.0

    def get_mode(self, dist_idx):
        return np.exp(self.distributions[dist_idx].mu - self.distributions[dist_idx].sigma**2)



## Dataset curation

In [ ]:
df = pd.read_json('../../data/construct_sequence_mut_rates.json')

df = df[df['sample']=='18DMS'].reset_index(drop=True)

signals = np.stack(df['mut_rates']).astype(float)
signals = np.nan_to_num(signals)

sequences = np.array([ list(seq) for seq in df['sequence']])
print("Number of sequences", len(sequences))


In [ ]:
## Checking where is signal in sequence

from scipy import interpolate

MS2_start = boundary['MS2'](df['sequence'][0])[0]
ROI_start = boundary['ROI'](df['sequence'][0])[0]
signal_before_ROI = signals[:, MS2_start:ROI_start]

signals_after_ROI = []
signals_ROI = []
seqs_ROI = []
for seq, signal in zip(sequences, signals):
    seq_str = ''.join(seq)
    signal_after_ROI = np.array(signal[boundary['TC2'](seq_str)[0]:boundary['LAH'](seq_str)[1] ]).astype(float)
    signals_after_ROI.append( np.nan_to_num(signal_after_ROI) )


    signal_ROI = np.array(signal[boundary['ROI'](seq_str)[0]:boundary['ROI'](seq_str)[1] ]).astype(float)
    seq_ROI = np.array(seq[boundary['ROI'](seq_str)[0]:boundary['ROI'](seq_str)[1] ])
    signals_ROI.append( np.nan_to_num(signal_ROI) )
    seqs_ROI.append(seq_ROI)
signals_after_ROI = np.array(signals_after_ROI)

streched_ROI_win = max([len(signal) for signal in signals_ROI])
streched_ROI_signals = np.zeros( (len(signals_ROI), streched_ROI_win) )
for i, signal in enumerate(signals_ROI):
    assert np.isnan(signal).any() == False, 'nan at {}'.format(i)
    f_signal = interpolate.interp1d(np.arange(len(signal)), signal)
    streched_ROI_signals[i] = f_signal(np.linspace(0, len(signal)-1, streched_ROI_win))

fig = go.Figure()
idx_start = ROI_start-MS2_start
fig.add_trace(go.Bar(
    name='Before ROI',
    x=np.arange(idx_start), y=np.mean(signal_before_ROI, axis=0),
    error_y=dict(type='data', array=np.std(signal_before_ROI, axis=0))
))

fig.add_trace(go.Bar(
    name='ROI (streched)',
    x=np.arange(idx_start, idx_start+streched_ROI_win), y=np.mean(streched_ROI_signals, axis=0),
    error_y=dict(type='data', array=np.std(streched_ROI_signals, axis=0))
))

fig.add_trace(go.Bar(
    name='After ROI',
    x=np.arange(idx_start+streched_ROI_win, idx_start+streched_ROI_win+20), y=np.mean(signals_after_ROI, axis=0),
    error_y=dict(type='data', array=np.std(signals_after_ROI, axis=0))
))


fig.update_layout(title='Average mutation rate accross all construct', barmode='group', xaxis_title='nucleotide position', yaxis_title='mutation rate')
fig.show()

print('Average std before ROI', np.mean(np.std(signal_before_ROI, axis=0)))
print('Average std in ROI', np.mean(np.std(streched_ROI_signals, axis=0)))
print('Average std after ROI', np.mean(np.std(signals_after_ROI, axis=0)))

# fig.write_html('/Users/alberic/Desktop/Pro/RouskinLab/projects/LaurenPaper/highthroughputcellularbiology/figs/average_signal.html')

In [ ]:
# Notes on sampels:
# 18 and 470
# 19 and 472

# 33, 36 are bad

# Results
# Across sample 18DMS
# Average std before ROI 0.009616456641118194
# Average std in ROI 0.019736332473797833
# Average std after ROI 0.011510102386244694

# Across all samples
# Average std before ROI 0.01080475276617488
# Average std in ROI 0.01984380795355725
# Average std after ROI 0.014755924434694043

In [ ]:
## Create dataset of signals for various sliding windows
mu_thresh = 0.08

def create_dataset(seq_list, signal_list, win_len):
    # The dataset is a dictionary, each key corresponding to one possible window of bases

    # Go over each sequence, extract sliding window array, and find matches with key window
    dataset = {}
    n_points = 0
    for sequence, signal in zip(seq_list, signal_list):
        idx_win = np.arange(len(sequence)-win_len+1)
        seq_triplets = sequence[idx_win[:, np.newaxis] + np.arange(win_len)]
        seq_triplets = seq_triplets[(seq_triplets[:, 1]=='A') | (seq_triplets[:, 1]=='C')]

        # for seq_window, seq_key in zip(seq_window_set, seq_window_set_key):
        for i, triplet in enumerate(seq_triplets):
            key = ''.join(triplet)

            if signal[i+1] < mu_thresh:
                if key in dataset:
                    dataset[key].append(signal[i+1])
                else:
                    dataset[key] = [signal[i+1]]
                
                n_points += 1
            

    print('Length of curated dataset', n_points)

    return dataset

gmm = GaussianMixture(n_components=2, max_iter=1000, covariance_type = 'full')
def fit_GMM(data):
    gmm.fit(np.array(data).reshape(-1,1))
    x_axis = np.linspace(min(data), max(data), 500)
    pdf = np.zeros_like(x_axis)
    for i_c in range(2):
        pdf += norm.pdf(x_axis, gmm.means_[i_c], np.sqrt(float(gmm.covariances_[i_c])))*gmm.weights_[i_c]

    return x_axis, pdf

win_len = 3
dataset = create_dataset(sequences, signals, win_len)
print("Length of full dataset", sequences.shape[0]*(sequences.shape[1]-(win_len-1)))

## Plots of histogram

In [ ]:
full_signal = np.concatenate(signals).ravel()
dataset['full'] = full_signal[full_signal < mu_thresh]

dist = logGMM()
dist.fit_logGMM(data=dataset['full'])

fig = go.Figure(
    go.Histogram(x=dataset['full'], histnorm='probability density', showlegend=False) )

x_axis = np.linspace(min(dataset['full']), max(dataset['full']), 500)
fig.add_trace( go.Scatter(x=x_axis, y=dist.get_pdf(x_axis), marker_color='red') )

fig.update_layout(
    title="Full dataset"+' ({:.0f} points) | Mu low:{:.2e} | Mu mid: {:.2e} | Mu high: {:.2e}'.format(
                            len(dataset['full']), 
                            dist.get_mode(0), dist.find_midpoint(), dist.get_mode(1)
                            ),
    xaxis_title="Mutation rate",
    yaxis_title="Probability density")
fig.update_yaxes(range=[0, 10*dist.get_pdf(dist.get_mode(1))[0]])

fig.show()
# fig.write_html('/Users/alberic/Desktop/Pro/RouskinLab/projects/LaurenPaper/highthroughputcellularbiology/figs/logGMM/full_histogram.html')

In [ ]:
dataset['A'] = []
dataset['C'] = []

for key, value in dataset.items():
    if len(key) == 3:
        if key[1] =='A':
            dataset['A'] += value

        if key[1] =='C':
            dataset['C'] += value

fig = make_subplots(rows=1, cols=2)

for i_b, key in enumerate(['A', 'C']):
    fig.add_trace(
        go.Histogram(x=dataset[key], histnorm='probability density', showlegend=False) , row=1, col=i_b+1)

    x_axis = np.linspace(min(dataset[key]), max(dataset[key]), 500)
    dist.fit_logGMM(data=dataset[key])
    
    fig.add_trace(go.Scatter(
            x=x_axis,
            y=dist.get_pdf(x_axis),
            showlegend=False,
            marker_color = px.colors.qualitative.D3[2]
        ), row=1, col=i_b+1)

    fig.update_xaxes(title_text=key+' ({:.0f} points) | Mu low:{:.2e} | Mu mid: {:.2e} | Mu high: {:.2e}'.format(
                            len(dataset[key]), 
                            dist.get_mode(0), dist.find_midpoint(), dist.get_mode(1)
                            ), row=1, col=i_b+1)
    fig.update_yaxes(range=[0, 10*dist.get_pdf(dist.get_mode(1))[0]], row=1, col=i_b+1)
fig.update_layout(
    height=500,
    width=1200,
    title_text="Histogram of DMS signals for A and C")
fig.show()
# fig.write_html('/Users/alberic/Desktop/Pro/RouskinLab/projects/LaurenPaper/highthroughputcellularbiology/figs/logGMM/A_C_histogram.html')

In [ ]:
mean = []
var = []
weight = []
fig = make_subplots(rows=len(dataset)//2, cols=2)

row_A = 1
row_C = 1
for i, key in enumerate(dataset.keys()):
    if len(key) == 3:

        col = 1 if key[1]=='A' else 2

        if key[1]=='A':
            row = row_A
            row_A +=1
            color= px.colors.qualitative.Plotly[1]
        else:
            row = row_C
            row_C +=1
            color= px.colors.qualitative.Plotly[0]

        fig.add_trace(
            go.Histogram(x=dataset[key], showlegend=False, histnorm='probability density', marker_color=color) , row=row, col=col)

        x_axis = np.linspace(min(dataset[key]), max(dataset[key]), 500)
        dist.fit_logGMM(data=dataset[key])

        # mean.append([mean for _, mean in sorted(zip(gmm.means_.T[0], gmm.means_.T[0])) ])
        # var.append([var for _, var in sorted(zip(gmm.means_.T[0], gmm.covariances_.T[0][0])) ])
        # weight.append([w for _, w in sorted(zip(gmm.means_.T[0], gmm.weights_)) ])
        
        fig.add_trace(go.Scatter(
            x=x_axis,
            y=dist.get_pdf(x_axis),
            showlegend=False,
            marker_color = px.colors.qualitative.D3[2]
        ), row=row, col=col)

        fig.update_xaxes(title_text=key+' ({:.0f} points) | Mu low:{:.2e} | Mu mid: {:.2e} | Mu high: {:.2e}'.format(
                            len(dataset[key]), 
                            dist.get_mode(0), dist.find_midpoint(), dist.get_mode(1)
                            ), row=row, col=col)

        fig.update_yaxes(range=[0, 10*dist.get_pdf(dist.get_mode(1))[0]], row=row, col=col)

fig.update_layout(
    height=4000,
    width=1400,
    title_text="Histogram of DMS signals for each triplet bases")
fig.show()

mean = np.array(mean)
var = np.array(var)
weight = np.array(weight)
# fig.write_html('/Users/alberic/Desktop/Pro/RouskinLab/projects/LaurenPaper/highthroughputcellularbiology/figs/logGMM/triplet_histogram.html')

In [ ]:
fig = go.Figure()

for mu, v, w in zip(mean, var, weight):
    fig.add_trace( go.Scatter(x = x_axis, y=norm.pdf(x_axis, mu[0], np.sqrt(v[0]))*w[0], marker_color='red', showlegend=False))
    fig.add_trace( go.Scatter(x = x_axis, y=norm.pdf(x_axis, mu[1], np.sqrt(v[1]))*w[1], marker_color='blue', showlegend=False))

# fig.update_yaxes(type='log', range=[0,2])
fig.show()

In [ ]:
fig = make_subplots(rows=3, cols=2)

for r, stat in enumerate([mean, var, weight]):

    for i in range(stat.shape[1]):
        fig.add_trace( go.Histogram(x=stat[:,i], showlegend=False) , row=r+1, col=i+1)

fig.update_layout(
    height=1000,
    width=1200)
fig.show()